In [1]:
import numpy as np

In [2]:
npzdata=np.load("data.npz")

In [3]:
data=npzdata['data']
labels=npzdata['labels']

In [4]:
data.shape

(249996, 5000)

In [5]:
import keras.preprocessing.sequence as sq

Using TensorFlow backend.


In [6]:
maxlen=150

In [7]:
data=sq.pad_sequences(data,maxlen=maxlen,padding='post',truncating='post',dtype='float64')

In [8]:
data=data.reshape(data.shape[0],data.shape[1],1)

In [9]:
from keras.utils import np_utils

In [10]:
dict_label={}

In [11]:
n=0

In [12]:
set_labels=list(set(labels))

In [13]:
for l in set_labels:
    dict_label[l]=n
    n+=1

In [14]:
labels_new=[]
for l in labels:
    labels_new.append(dict_label[l])

label_1=np_utils.to_categorical(labels_new)

In [15]:
def generateData(data,labels,batch_size=32):
    nb_instances = data.shape[0]
    nb_classes = labels.shape[1]
    sample_shape = data[0].shape
    batch_data_shape = tuple([batch_size] + list(sample_shape))
    batch_label_shape = (batch_size, nb_classes)
    # Infinite loop
    while True:
        # Generate an exploration order
        indices = np.arange(nb_instances)

        np.random.shuffle(indices)

        # Generate batches
        imax = int(len(indices) / batch_size)
        for i in range(imax):
            # Form a batch
            x = np.empty(batch_data_shape)
            y = np.empty(batch_label_shape)
            for j, k in enumerate(indices[i * batch_size: (i + 1) * batch_size]):
                x[j] = data[k]
                y[j] = labels[k]
            if x.shape != batch_data_shape:
                print(x.shape)
                exit(0)
            yield x, y

In [16]:
from keras.layers.core import Dense, Dropout
from keras.layers.recurrent import GRU
from keras.layers import Input
from keras.models import Sequential
from keras.optimizers import RMSprop

In [17]:
model=Sequential()
input_length=maxlen
input_dim=1

In [18]:
model.add(GRU(input_shape=(input_length,input_dim),units=128,activation='tanh',recurrent_activation='hard_sigmoid',return_sequences=True,dropout=0.2))

In [20]:
model.add(GRU(units=128,activation='tanh',recurrent_activation='hard_sigmoid',return_sequences=False,dropout=0.2))

In [21]:
nb_classes=label_1.shape[1]
model.add(Dense(units=nb_classes,activation='softmax'))
metrics=['accuracy']
optimizer=RMSprop(lr=0.001,decay=0.0)
model.compile(loss="categorical_crossentropy",optimizer=optimizer,metrics=metrics)

In [22]:
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 150, 128)          49920     
_________________________________________________________________
gru_2 (GRU)                  (None, 128)               98688     
_________________________________________________________________
dense_1 (Dense)              (None, 101)               13029     
Total params: 161,637
Trainable params: 161,637
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
from sklearn import model_selection
X_train, X_test, y_train, y_test =model_selection.train_test_split(data, label_1, test_size=0.33)
trainGen=generateData(X_train,y_train)
valGen=generateData(X_test,y_test)

In [24]:
history=model.fit_generator(generator=trainGen,steps_per_epoch=X_train.shape[0]//32,validation_data=valGen,validation_steps=X_test.shape[0]//32,epochs=5)

Epoch 1/5


InternalError: Blas GEMM launch failed : a.shape=(32, 128), b.shape=(128, 128), m=32, n=128, k=128
	 [[Node: gru_1/while/MatMul_3 = MatMul[T=DT_FLOAT, _class=["loc:@training/RMSprop/gradients/gru_1/while/MatMul_3_grad/MatMul"], transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](gru_1/while/Switch_3:1, gru_1/while/MatMul_3/Enter)]]
	 [[Node: loss/mul/_83 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_2846_loss/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]